In [84]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
import os

In [85]:
SampleRate = 256

In [86]:
HBLDetections = scipy.io.loadmat('./../Ascertain_Feature Extraction_Implementations-V1/ECG/HBLDetections.mat')
#HBLDetections = HBLDetections['ClipHBLs']

In [87]:
PreferableChannel_done = scipy.io.loadmat('./../Ascertain_Feature Extraction_Implementations-V1/ECG/PreferableChannel_done.mat')
#HBLDetections = HBLDetections['ClipHBLs']

In [88]:
HBLDetections['ClipHBLs'][0,0][0,0][0]

array([  0.15625   ,   0.8359375 ,   1.53125   ,   2.24609375,
         2.98046875,   3.6640625 ,   4.3671875 ,   5.1015625 ,
         5.88671875,   6.71484375,   7.515625  ,   8.29296875,
         9.109375  ,   9.94140625,  10.76953125,  11.5390625 ,
        12.265625  ,  13.0078125 ,  13.78515625,  14.60546875,
        15.4453125 ,  16.21875   ,  17.        ,  17.81640625,
        18.640625  ,  19.44921875,  20.19140625,  20.96875   ,
        21.76171875,  22.5703125 ,  23.32421875,  24.08203125,
        24.890625  ,  25.7421875 ,  26.5546875 ,  27.34375   ,
        28.171875  ,  29.02734375,  29.86328125,  30.61328125,
        31.40625   ,  32.1796875 ,  32.93359375,  33.62890625,
        34.34765625,  35.1328125 ,  35.96875   ,  36.80859375,
        37.60546875,  38.44921875,  39.30859375,  40.12890625,
        40.89453125,  41.69140625,  42.50390625,  43.296875  ,
        44.03515625,  44.79296875,  45.59375   ,  46.453125  ,
        47.3046875 ,  48.10546875,  48.94921875,  49.82

In [89]:
PreferableChannel_done

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Aug 08 19:16:12 2016',
 '__version__': '1.0',
 '__globals__': [],
 'ClipID': array([[36]], dtype=uint8),
 'ECGHREval42': array([[3, 6, 1, ..., 2, 4, 2],
        [3, 6, 1, ..., 2, 4, 2],
        [3, 6, 3, ..., 2, 4, 2],
        ...,
        [2, 6, 6, ..., 2, 4, 2],
        [1, 6, 6, ..., 3, 4, 3],
        [2, 6, 6, ..., 3, 4, 3]], dtype=uint8),
 'Ev': array([[2],
        [2],
        [3],
        [3]], dtype=uint8),
 'Preferable': array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]], dtype=uint8),
 'UserID': array([[42]], dtype=uint8)}

In [90]:
ECGFeatures = [[] for i in range(1, 68)]
#ECGFeatures = np.array(ECGFeatures)
str(10).zfill(2)

'10'

In [106]:
Preferable = PreferableChannel_done['Preferable']
ClipHBLs = HBLDetections['ClipHBLs']

signal = 0
for UserID in range(10, 11): #68
    print('USER ID ' + str(UserID))
    
    # Create features and failures lists for each user
    ECGFeatures = [[] for i in range(1, 68)]
    ECGFailures = [[] for i in range(1, 68)]
    print(UserID)
    
    for ClipID in range(1, 2): #37
        
        # check if data exists for this user and this clip
        dataPath = './../ASCERTAIN_Raw/ECGData/Movie_P' + str(UserID).zfill(2) + '/ECG_Clip' + str(ClipID) + '.mat'
        if os.path.isfile(dataPath):
           
            signal = scipy.io.loadmat(dataPath)
            Data_ECG = signal['Data_ECG']
            #print(Data_ECG)
            
            if UserID < 11:
                ECG2 = np.array([Data_ECG[2, :], Data_ECG[3, :]])
            else:
                ECG2 = np.array([Data_ECG[5, :], Data_ECG[6, :]])
                
            #print(ECG2)
            
            if (Data_ECG.shape[0] >= 50*SampleRate):
                
                GoodChannel = Preferable[UserID - 1, ClipID - 1]
                HeartBeatsL = ClipHBLs[ClipID - 1, UserID - 1]
                #print(HeartBeatsL)
                ECG2 = ECG2[len(ECG2) - 50*SampleRate:len(ECG2), GoodChannel]
                HeartBeatsL = HeartBeatsL[:,GoodChannel - 1]
                print(HeartBeatsL)
                
                if (HeartBeatsL[-1][0].any() > 50):
                    HeartBeatsL = HeartBeatsL[HeartBeatsL > HeartBeatsL[-1] - 50]
                    HeartBeatsL = HeartBeatsL - min(HeartBeatsL[0])
                    
                #print(HeartBeatsL)  
                IBI = np.diff(HeartBeatsL)
                HR = 60 / IBI
                HRV = HR[1:] - HR[:-1]
                IBIFeatures = [ np.mean(IBI),
                                np.std(IBI), 
                                kurtosis(IBI),
                                skew(IBI), 
                                np.sum(IBI > (np.mean(IBI) + np.std(IBI))) / len(IBI), 
                                np.sum(IBI < (np.mean(IBI) - np.std(IBI))) / len(IBI)]
                
                
            else:
                print('Shorter than 50 secs')
                Features = np.zeros(32) + np.nan
                ECGFailures[UserID].append(ClipID)
        

USER ID 10
10
[array([[ 0.3125    ,  0.94921875,  1.609375  ,  2.28515625,  3.05078125,
         3.8828125 ,  4.69921875,  5.44140625,  6.203125  ,  6.9921875 ,
         7.79296875,  8.55859375,  9.27734375, 10.01953125, 10.7578125 ,
        11.51171875, 12.23046875, 12.9453125 , 13.66796875, 14.3984375 ,
        15.1328125 , 15.84765625, 16.57421875, 17.328125  , 18.11328125,
        18.8828125 , 19.625     , 20.39453125, 21.1796875 , 21.96875   ,
        22.72265625, 23.4921875 , 24.2734375 , 25.0390625 , 25.76171875,
        26.515625  , 27.29296875, 28.07421875, 28.8359375 , 29.60546875,
        30.390625  , 31.16796875, 31.8984375 , 32.6484375 , 33.421875  ,
        34.21484375, 34.98828125, 35.73046875, 36.50390625, 37.2890625 ,
        38.05078125, 38.7734375 , 39.52734375, 40.30859375, 41.09765625,
        41.82421875, 42.57421875, 43.34765625, 44.1171875 , 44.8359375 ,
        45.57421875, 46.33984375, 47.12109375, 47.86328125, 48.61328125,
        49.3671875 , 50.1328125 , 50

C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in long_scalars


TypeError: numpy boolean subtract, the `-` operator, is deprecated, use the bitwise_xor, the `^` operator, or the logical_xor function instead.

In [ ]:
HeartBeatsL[0].shape

In [ ]:
signal['V_ECG']